# 618 WorkFlow Gerencial - SOLO TREND Features (Experimento de Ablación)

### 618.1 Objetivo

**EXPERIMENTO 618: SOLO FEATURES DE TENDENCIA (TREND)**

Este experimento es parte de un estudio de ablación para medir el impacto específico
de las features de TENDENCIA (trend_3, trend_6) en el modelo predictivo.

**HIPÓTESIS:**
Según el análisis de feature importance del experimento WF616 (completo), las features
de TREND contribuyen con aproximadamente 61.2% de la ganancia del modelo.

**FEATURES INCLUIDAS:**
- ✅ Variables originales (~29)
- ✅ FE intrames (kmes, mpayroll_sobre_edad)
- ✅ **TRENDS**: Pendientes de regresión lineal sobre ventanas de 3 y 6 meses (~58 features)

**FEATURES EXCLUIDAS:**
- ❌ Lags (1,2,3,6)
- ❌ Deltas (1,2,3)
- ❌ Rolling statistics (mean, max, min, sd)
- ❌ Ratios históricos
- ❌ Volatilidad (CV, range_norm)

**OBJETIVO DEL EXPERIMENTO:**
Validar si SOLO las features de TREND (sin ninguna otra FE histórica) son suficientes
para alcanzar ~60-70% de la ganancia del modelo completo (WF616).

**COMPARACIÓN ESPERADA:**
- WF616 (completo): $13.6M (689 features) → 100%
- WF618 (solo TREND): $8-10M (~87 features) → 60-75% esperado
- WF617 (sin TREND): $5-7M (631 features) → 37-52% esperado

**IMPLICANCIA:**
Si WF618 logra >70% de ganancia con solo 13% de las features, demostraría que:
1. TREND es el componente más crítico del modelo
2. Es posible simplificar significativamente el pipeline de FE
3. Las otras features aportan valor marginal (~30-40%)

#### 618.2 Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
* Crear las carpetas en el Google Drive
* "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria kaggle de Python
* Bajar el **dataset_pequeno** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab
* Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab

In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget $webfiles/$archivo -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp $destino_bucket/$archivo $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget $webfiles/$archivo -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp $destino_bucket/$archivo $destino_local/$archivo
fi

## 618.3 Workflow con 5 Semillas - SOLO TREND

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

In [ ]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

#### Parametros Globales
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [ ]:
PARAM_GLOBAL <- list()
PARAM_GLOBAL$experimento_base <- 6180  # Experimento 618: SOLO TREND
PARAM_GLOBAL$dataset <- "gerencial_competencia_2025.csv.gz"

# Vector de 5 semillas diferentes (mismas que WF616 para comparación)
PARAM_GLOBAL$semillas <- c(153929, 838969, 922081, 795581, 194609)

# Lista para almacenar resultados de todas las semillas
resultados_totales <- list()

## 618.3 Loop Principal - Iteración Automática sobre las 5 Semillas

Este loop ejecuta todo el workflow completo para cada una de las 5 semillas de manera automática.

**IMPORTANTE:** Este experimento SOLO incluye features de TREND (trend_3, trend_6).

In [ ]:
# Loop automático sobre todas las semillas
for (seed_idx in 1:length(PARAM_GLOBAL$semillas)) {

  cat("\n\n========================================\n")
  cat("PROCESANDO SEMILLA ", seed_idx, " de ", length(PARAM_GLOBAL$semillas), "\n")
  cat("Semilla: ", PARAM_GLOBAL$semillas[seed_idx], "\n")
  cat("========================================\n\n")

  # Inicializar PARAM para esta semilla
  PARAM <- list()
  PARAM$semilla_primigenia <- PARAM_GLOBAL$semillas[seed_idx]
  PARAM$experimento <- PARAM_GLOBAL$experimento_base + seed_idx - 1
  PARAM$dataset <- PARAM_GLOBAL$dataset
  PARAM$out <- list()
  PARAM$out$lgbm <- list()

  # ===================================================================
  # Carpeta del Experimento
  # ===================================================================
  setwd("/content/buckets/b1/exp")
  experimento_folder <- paste0("WF", PARAM$experimento, "_seed", seed_idx, "_ONLY_TREND")
  dir.create(experimento_folder, showWarnings=FALSE)
  setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))
  
  cat("Carpeta de trabajo: ", experimento_folder, "\n")
  
  inicio_seed <- Sys.time()

  # ===================================================================
  # Preprocesamiento del dataset
  # ===================================================================
  
  cat("\nCargando dataset...\n")
  
  # Lectura del dataset
  dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

  cat("Dataset cargado:", nrow(dataset), "filas x", ncol(dataset), "columnas\n")

  # Catastrophe Analysis (202006)
  cat("Aplicando Catastrophe Analysis...\n")
  dataset[ foto_mes==202006, internet:=NA]
  dataset[ foto_mes==202006, mrentabilidad:=NA]
  dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
  dataset[ foto_mes==202006, mcomisiones:=NA]
  dataset[ foto_mes==202006, mactivos_margen:=NA]
  dataset[ foto_mes==202006, mpasivos_margen:=NA]
  dataset[ foto_mes==202006, mcuentas_saldo:=NA]
  dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
  dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
  dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
  dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
  dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

  cat("✓ Catastrophe Analysis aplicado (13 variables)\n")
  
  # ===================================================================
  # Feature Engineering INTRAMES (mínimo)
  # ===================================================================
  
  cat("\nFeature Engineering intrames...\n")
  
  # kmes (mes del año)
  dataset[, kmes := foto_mes %% 100]
  
  # Ratio mpayroll / cliente_edad
  dataset[, mpayroll_sobre_edad := ifelse(cliente_edad > 0, mpayroll / cliente_edad, NA)]
  
  cat("✓ FE intrames completado: kmes, mpayroll_sobre_edad\n")
  cat("Variables actuales:", ncol(dataset), "\n")
  
  # ===================================================================
  # ⭐ FEATURE ENGINEERING: SOLO TRENDS ⭐
  # ===================================================================
  
  cat("\n========================================\n")
  cat("FEATURE ENGINEERING: SOLO TRENDS\n")
  cat("========================================\n\n")
  
  inicio_fe <- Sys.time()
  
  # Ordenar dataset por cliente y mes (fundamental para TREND)
  setorder(dataset, numero_de_cliente, foto_mes)
  
  # Definir las columnas sobre las que se calculará TREND
  cols_lagueables <- copy( setdiff(
      colnames(dataset),
      c("numero_de_cliente", "foto_mes", "clase_ternaria", "kmes", "mpayroll_sobre_edad")
  ) )
  
  cat("Número de variables base para TREND:", length(cols_lagueables), "\n\n")
  
  # ===================================================================
  # TÉCNICA: TENDENCIAS (Slope/pendiente de regresión lineal)
  # ===================================================================
  # Justificación: La tendencia captura la DIRECCIÓN del cambio en el tiempo.
  # Calculamos la pendiente de una regresión lineal sobre ventanas de 3 y 6 meses.
  # Una pendiente positiva indica crecimiento, negativa indica decrecimiento.
  # 
  # Ejemplo:
  #   mcuentas_saldo: [$80K, $50K, $20K] → trend_3 = -$30K/mes (¡cayendo rápido!)
  #   mtarjeta_consumo: [$10K, $12K, $15K] → trend_3 = +$2.5K/mes (creciendo)
  #
  # TREND es más robusto que deltas porque:
  # - Considera TODA la ventana temporal (no solo 2 puntos)
  # - Filtra fluctuaciones aleatorias
  # - Captura el MOMENTUM del cambio
  
  cat("Generando TENDENCIAS (slopes lineales)...\n")
  cat("Esto puede tardar varios minutos...\n\n")
  
  # Función para calcular pendiente de regresión lineal
  calc_slope <- function(y) {
      if(all(is.na(y))) return(NA_real_)
      x <- 1:length(y)
      valid <- !is.na(y)
      if(sum(valid) < 2) return(NA_real_)
      tryCatch({
          coef(lm(y[valid] ~ x[valid]))[2]
      }, error = function(e) NA_real_)
  }
  
  for (vcol in cols_lagueables)
  {
      # Tendencia sobre 3 meses
      # Usa frollapply para calcular pendiente en ventana móvil de 3 meses
      dataset[, paste0(vcol, "_trend_3") := frollapply(
          x = get(vcol),
          n = 3,
          FUN = calc_slope,
          align = "right"
      ), by = numero_de_cliente]
      
      # Tendencia sobre 6 meses
      # Captura patrones de largo plazo y estacionalidad semestral
      dataset[, paste0(vcol, "_trend_6") := frollapply(
          x = get(vcol),
          n = 6,
          FUN = calc_slope,
          align = "right"
      ), by = numero_de_cliente]
  }
  
  fin_fe <- Sys.time()
  tiempo_fe <- as.numeric(difftime(fin_fe, inicio_fe, units = "mins"))
  
  total_trends <- length(cols_lagueables) * 2
  
  cat("\n========================================\n")
  cat("RESUMEN FEATURE ENGINEERING\n")
  cat("========================================\n")
  cat("Variables base:", length(cols_lagueables), "\n")
  cat("\nNuevas features TREND generadas:\n")
  cat("  - Tendencias (3 meses):", length(cols_lagueables), "\n")
  cat("  - Tendencias (6 meses):", length(cols_lagueables), "\n")
  cat("\nTOTAL NUEVAS TRENDS:", total_trends, "\n")
  cat("TOTAL FEATURES EN DATASET:", ncol(dataset), "\n")
  cat("Tiempo de generación:", round(tiempo_fe, 1), "minutos\n")
  cat("\n⚠️ NO se incluyeron: lags, deltas, rolling, ratios, volatilidad\n")
  cat("⚠️ SOLO TRENDS para medir su impacto aislado\n")
  cat("========================================\n\n")

  # ===================================================================
  # Modelado - Training Strategy
  # ===================================================================
  
  cat("Configurando Training Strategy...\n")
  
  PARAM$trainingstrategy <- list()
  PARAM$trainingstrategy$validate <- c(202105)
  
  PARAM$trainingstrategy$training <- c(
    202104, 202103, 202102, 202101,
    202012, 202011, 202010, 202009, 202008, 202007,
    202006, 202005
  )
  
  PARAM$trainingstrategy$training_pct <- 1.0
  PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

  dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

  campos_buenos <- copy( setdiff(
      colnames(dataset), c("clase_ternaria","clase01","azar"))
  )

  set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
  dataset[, azar:=runif(nrow(dataset))]
  
  dataset[, fold_train := foto_mes %in% PARAM$trainingstrategy$training &
      (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
       azar < PARAM$trainingstrategy$training_pct ) ]
  
  cat("Train set:", nrow(dataset[fold_train == TRUE]), "filas\n")
  cat("Validation set:", nrow(dataset[foto_mes %in% PARAM$trainingstrategy$validate]), "filas\n")
  cat("Features para modelado:", length(campos_buenos), "\n\n")
  
  if( !require("lightgbm")) install.packages("lightgbm")
  require("lightgbm")
  
  dtrain <- lgb.Dataset(
    data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
    label= dataset[fold_train == TRUE, clase01],
    free_raw_data= FALSE
  )

  dvalidate <- lgb.Dataset(
    data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
    label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
    free_raw_data= FALSE
  )

  # ===================================================================
  # Hyperparameter Tuning
  # ===================================================================
  
  cat("\nIniciando Hyperparameter Tuning...\n")
  
  if(!require("DiceKriging")) install.packages("DiceKriging")
  require("DiceKriging")
  
  if(!require("mlrMBO")) install.packages("mlrMBO")
  require("mlrMBO")

  PARAM$hipeparametertuning <- list()
  PARAM$hipeparametertuning$num_iterations <- 100
  PARAM$lgbm <- list()
  
  PARAM$lgbm$param_fijos <- list(
    objective= "binary",
    metric= "auc",
    first_metric_only= TRUE,
    boost_from_average= TRUE,
    feature_pre_filter= FALSE,
    verbosity= -100,
    seed= PARAM$semilla_primigenia,
    max_bin= 31,
    learning_rate= 0.03,
    feature_fraction= 0.5,
    num_iterations= 2048,
    early_stopping_rounds= 200
  )
  
  PARAM$hipeparametertuning$hs <- makeParamSet(
    makeIntegerParam("num_leaves", lower = 2L, upper = 256L),
    makeIntegerParam("min_data_in_leaf", lower = 2L, upper = 8192L)
  )

  EstimarGanancia_AUC_lightgbm <- function(x) {
  
    param_completo <- modifyList(PARAM$lgbm$param_fijos, x)
  
    modelo_train <- lgb.train(
      data= dtrain,
      valids= list(valid = dvalidate),
      eval= "auc",
      param= param_completo,
      verbose= -100
    )
  
    AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]
    attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)
  
    rm(modelo_train)
    gc(full= TRUE, verbose= FALSE)
  
    return(AUC)
  }

  configureMlr(show.learner.output = FALSE)
  
  obj.fun <- makeSingleObjectiveFunction(
      fn= EstimarGanancia_AUC_lightgbm,
      minimize= FALSE,
      noisy= FALSE,
      par.set= PARAM$hipeparametertuning$hs,
      has.simple.signature= FALSE
  )
  
  ctrl <- makeMBOControl(
      save.on.disk.at.time= 600,
      save.file.path= "HT.RDATA"
  )
  
  ctrl <- setMBOControlTermination(
      ctrl,
      iters= PARAM$hipeparametertuning$num_iterations
  )
  
  ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())
  
  surr.km <- makeLearner(
      "regr.km",
      predict.type= "se",
      covtype= "matern3_2",
      control= list(trace = TRUE)
  )

  if (!file.exists("HT.RDATA")) {
    bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
  } else {
    bayesiana_salida <- mboContinue("HT.RDATA")
  }

  tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
  setorder(tb_bayesiana, -y, -num_iterations)
  
  fwrite( tb_bayesiana,
    file="BO_log.txt",
    sep="\t"
  )
  
  PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
    1,
    setdiff(colnames(tb_bayesiana),
      c("y","dob","eol","error.message","exec.time","ei","error.model",
        "train.time","prop.type","propose.time","se","mean","iter")),
    with= FALSE
  ]
  
  mejor_auc <- tb_bayesiana[1, y]
  
  cat("\nMejor AUC encontrado:", mejor_auc, "\n")
  cat("Mejores hiperparámetros:\n")
  print(PARAM$out$lgbm$mejores_hiperparametros)

  # ===================================================================
  # Produccion
  # ===================================================================
  
  cat("\nEntrenando modelo final...\n")
  
  PARAM$trainingstrategy$final_train <- c(
    202105, 202104, 202103, 202102, 202101,
    202012, 202011, 202010, 202009, 202008, 202007,
    202006, 202005
  )
  
  dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]
  
  dfinal_train <- lgb.Dataset(
    data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
    label= dataset[fold_final_train == TRUE, clase01],
    free_raw_data= FALSE
  )

  fijos <- copy(PARAM$lgbm$param_fijos)
  fijos$num_iterations <- NULL
  fijos$early_stopping_rounds <- NULL
  
  param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

  set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
  
  final_model <- lgb.train(
    data= dfinal_train,
    param= param_final,
    verbose= -100
  )

  lgb.save(final_model, "modelo.txt")

  tb_importancia <- as.data.table(lgb.importance(final_model))
  fwrite( tb_importancia,
    file= "impo.txt",
    sep= "\t"
  )

  # ===================================================================
  # Scoring
  # ===================================================================
  
  cat("\nGenerando predicciones...\n")
  
  PARAM$trainingstrategy$future <- c(202107)
  dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

  prediccion <- predict(
    final_model,
    data.matrix(dfuture[, campos_buenos, with= FALSE])
  )

  tb_prediccion <- dfuture[, list(numero_de_cliente)]
  tb_prediccion[, prob := prediccion]
  
  fwrite(tb_prediccion,
    file= "prediccion.txt",
    sep= "\t"
  )

  # ===================================================================
  # Curva de Ganancia
  # ===================================================================
  
  tb_prediccion[, clase_ternaria := dfuture$clase_ternaria ]
  tb_prediccion[, ganancia := -3000.0 ]
  tb_prediccion[clase_ternaria=="BAJA+2", ganancia := 117000.0 ]
  
  setorder( tb_prediccion, -prob )
  tb_prediccion[, gan_acum := cumsum(ganancia)]
  
  tb_prediccion[,
    gan_suavizada := frollmean(
      x= gan_acum,
      n= 400,
      align= "center",
      na.rm= TRUE,
      hasNA= TRUE
    )
  ]

  resultado <- list()
  resultado$ganancia_suavizada_max <- max( tb_prediccion$gan_suavizada, na.rm=TRUE )
  options(digits= 8)
  resultado$envios <- which.max( tb_prediccion$gan_suavizada)
  resultado$semilla <- PARAM$semilla_primigenia
  resultado$seed_idx <- seed_idx
  resultado$mejor_auc <- mejor_auc
  
  fin_seed <- Sys.time()
  resultado$duracion_min <- as.numeric(difftime(fin_seed, inicio_seed, units = "mins"))

  fwrite( tb_prediccion,
    file= "ganancias.txt",
    sep= "\t"
  )

  tb_prediccion[, envios:= .I]
  
  pdf("curva_de_ganancia.pdf")
  
  plot(
    x= tb_prediccion$envios,
    y= tb_prediccion$gan_acum,
    type= "l",
    col= "gray",
    xlim= c(0, 6000),
    ylim= c(0, 15000000),
    main= paste0("Seed ", seed_idx, " (ONLY TREND) - Gan= ", 
                 formatC(resultado$ganancia_suavizada_max, format="f", big.mark=",", digits=0), 
                 " envios= ", resultado$envios),
    xlab= "Envios",
    ylab= "Ganancia",
    panel.first= grid()
  )
  
  dev.off()

  if( !require("yaml")) install.packages("yaml")
  require("yaml")
  
  PARAM$resultado <- resultado
  
  write_yaml( PARAM, file="PARAM.yml")

  # ===================================================================
  # Guardar resultado y limpiar para siguiente iteración
  # ===================================================================
  
  if(!exists("resultados_totales")) resultados_totales <- list()
  resultados_totales[[seed_idx]] <- resultado

  rm(dataset, dtrain, dvalidate, dfinal_train, final_model, tb_prediccion)
  gc(full=TRUE, verbose=FALSE)

  cat("\n========================================\n")
  cat("Semilla ", seed_idx, " completada exitosamente\n")
  cat("Ganancia: ", formatC(resultado$ganancia_suavizada_max, format="f", big.mark=",", digits=0), "\n")
  cat("Envios: ", resultado$envios, "\n")
  cat("AUC: ", round(resultado$mejor_auc, 6), "\n")
  cat("Duración: ", round(resultado$duracion_min, 1), " minutos\n")
  cat("========================================\n\n")

} # Fin del loop sobre las semillas

cat("\n\n***************************************\n")
cat("TODAS LAS SEMILLAS PROCESADAS\n")
cat("***************************************\n")

## Resumen de Resultados de Todas las Semillas (SOLO TREND)

In [ ]:
# Crear tabla resumen
setwd("/content/buckets/b1/exp")

tb_resumen <- data.table(
  seed_idx = sapply(resultados_totales, function(x) x$seed_idx),
  semilla = sapply(resultados_totales, function(x) x$semilla),
  ganancia = sapply(resultados_totales, function(x) x$ganancia_suavizada_max),
  envios = sapply(resultados_totales, function(x) x$envios),
  mejor_auc = sapply(resultados_totales, function(x) x$mejor_auc),
  duracion_min = sapply(resultados_totales, function(x) x$duracion_min)
)

# Agregar estadísticas
tb_resumen[, rank := rank(-ganancia)]
tb_resumen[, status := "OK"]

cat("\n\n========================================\n")
cat("RESUMEN FINAL DE LAS 5 SEMILLAS\n")
cat("EXPERIMENTO 618: SOLO TREND FEATURES\n")
cat("========================================\n\n")
print(tb_resumen)

cat("\nESTADÍSTICAS:\n")
cat("Ganancia promedio: $", formatC(mean(tb_resumen$ganancia), format="f", big.mark=",", digits=0), "\n")
cat("Ganancia máxima: $", formatC(max(tb_resumen$ganancia), format="f", big.mark=",", digits=0), "\n")
cat("Ganancia mínima: $", formatC(min(tb_resumen$ganancia), format="f", big.mark=",", digits=0), "\n")
cat("Desviación estándar: $", formatC(sd(tb_resumen$ganancia), format="f", big.mark=",", digits=0), "\n")
cat("Coeficiente de variación: ", round(sd(tb_resumen$ganancia)/mean(tb_resumen$ganancia)*100, 2), "%\n")
cat("\nAUC promedio: ", round(mean(tb_resumen$mejor_auc), 6), "\n")
cat("Envíos promedio: ", round(mean(tb_resumen$envios), 0), "\n")
cat("Duración promedio: ", round(mean(tb_resumen$duracion_min), 1), " minutos\n")
cat("\nMejor semilla: ", tb_resumen[rank==1, semilla], " (seed_idx ", tb_resumen[rank==1, seed_idx], ")\n")

# Guardar resumen
fwrite(tb_resumen, 
  file=paste0("resumen_only_trend_exp", PARAM_GLOBAL$experimento_base, ".txt"),
  sep="\t"
)

# Guardar objeto completo
saveRDS(resultados_totales, 
  file=paste0("resultados_only_trend_exp", PARAM_GLOBAL$experimento_base, ".rds")
)

cat("\nArchivos guardados:\n")
cat("- resumen_only_trend_exp", PARAM_GLOBAL$experimento_base, ".txt\n")
cat("- resultados_only_trend_exp", PARAM_GLOBAL$experimento_base, ".rds\n")
cat("\nCada semilla tiene su carpeta individual con resultados detallados.\n")

cat("\n========================================\n")
cat("FEATURE ENGINEERING APLICADO:\n")
cat("========================================\n")
cat("✅ Variables base (~29)\n")
cat("✅ FE intrames (kmes, mpayroll_sobre_edad)\n")
cat("✅ TRENDS: Pendientes lineales (trend_3, trend_6) ~", length(cols_lagueables) * 2, " features\n")
cat("\n❌ NO se incluyeron:\n")
cat("   - Lags (1,2,3,6)\n")
cat("   - Deltas (1,2,3)\n")
cat("   - Rolling statistics\n")
cat("   - Ratios históricos\n")
cat("   - Volatilidad\n")
cat("\nTOTAL FEATURES: ~", length(cols_lagueables) + 2 + length(cols_lagueables) * 2, "\n")
cat("========================================\n")

cat("\n========================================\n")
cat("COMPARACIÓN CON OTROS EXPERIMENTOS:\n")
cat("========================================\n")
cat("WF616 (completo):  ~$13.6M (689 features)  100%\n")
cat("WF618 (solo TREND):  ~$", formatC(mean(tb_resumen$ganancia), format="f", big.mark=",", digits=0), 
    " (~87 features)  ", round(mean(tb_resumen$ganancia)/13600000*100, 0), "%\n")
cat("WF617 (sin TREND):  ~$5.7M (631 features)  42%\n")
cat("z610 (básico):  ~$6.0M (158 features)  44%\n")
cat("========================================\n")

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")